In [23]:
import json
import math
import os
import random
from pathlib import Path

import numpy as np
import pandas as pd

In [24]:
random.seed(4)

In [25]:
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks-in-python
def chunks(l, n):
    """Yield successive n-sized chunks from list l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [26]:
paths = []
with open('test.json') as json_file:
    paths = json.load(json_file)

list.sort(paths) # Ascending order

In [27]:
# Number of files a node will read from at a time
per = 4
num_nodes = 8

In [29]:
# assignment_non
# 8 nodes, 4 chosen by characteristics, rest randomly drawn
non_assignment = []
non_assignment.append([["household_031.feather", "household_127.feather", "household_175.feather", "household_197.feather"]])
non_assignment.append([["household_030.feather", "household_039.feather", "household_044.feather", "household_092.feather"]])
non_assignment.append([["household_009.feather", "household_019.feather", "household_161.feather", "household_195.feather"]])
non_assignment.append([["household_020.feather", "household_078.feather", "household_124.feather", "household_154.feather"]])

used = ["031", "127", "175", "197", 
        "030", "039", "092", "044", 
        "009", "019", "161", "195",
        "078", "124", "020", "154"]

remaining_paths = []
for p in paths:
    good = True
    for u in used:
        if u in p:
            good = False
            break
    if good:
        remaining_paths.append(p)

assert len(remaining_paths) == (200 - 4 * 4)


picked_households = list(
    chunks(
        random.sample(remaining_paths, (num_nodes - len(non_assignment)) * per * 1),
        num_nodes - len(non_assignment)))

i = 0
while len(non_assignment) < num_nodes:
    non_assignment.append(list(chunks(picked_households[i], per)))
    i = i + 1

with open('assignment_non.json', 'w') as outfile:
    json.dump(non_assignment, outfile)

In [28]:
# assignment_iid
# 8 nodes, shuffle the non-IID assignment
iid_assignment = []

for node in non_assignment:
    for hs in node:
        for h in hs:
            iid_assignment.append(h)
            
random.shuffle(iid_assignment)
iid_assignment = [ [l] for l in list(chunks(iid_assignment, per))]

with open('assignment_iid.json', 'w') as outfile:
    json.dump(iid_assignment, outfile)